In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/dacon/CV/도배하자유형분류경진대회

/content/drive/MyDrive/Colab Notebooks/dacon/CV/도배하자유형분류경진대회


In [ ]:
!nvidia-smi #gpu사양확인

Tue May  9 09:16:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 클래스개수확인

In [ ]:
import random
import pandas as pd
import numpy as np
import glob
import cv2

from torch.utils.data import Dataset, DataLoader
import cv2
import os 
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 augmentation
import glob
import albumentations as A
import albumentations.pytorch
from albumentations.pytorch.transforms import ToTensorV2 # albumentations 텐서화 함수
import torch.nn as nn

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정
torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)
device

'cuda'

# Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}
criterion = nn.CrossEntropyLoss().to(device) # cost function

# Fixted RandomSeed

In [ ]:
# def seed_everything(seed):
#    random.seed(seed)
#    os.environ['PYTHONHASHSEED'] = str(seed)
#    np.random.seed(seed)
#    torch.manual_seed(seed)
#    torch.cuda.manual_seed(seed)
#    torch.backends.cudnn.deterministic = True
#    torch.backends.cudnn.benchmark = True

#seed_everything(CFG['SEED']) # Seed 고정

# Data Pre-processing

In [ ]:
all_img_list = glob.glob('./train/*/*')
# all_img_list

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[2])
df

,img_path,label
0,./train/가구수정/7.png,가구수정
1,./train/가구수정/3.png,가구수정
2,./train/가구수정/10.png,가구수정
3,./train/가구수정/6.png,가구수정
4,./train/가구수정/5.png,가구수정
...,...,...
3452,./train/꼬임/55.png,꼬임
3453,./train/꼬임/126.png,꼬임
3454,./train/꼬임/187.png,꼬임
3455,./train/꼬임/51.png,꼬임


In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

# Label-Encoding

In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

# CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transforms = A.Compose([
    A.Resize(224,224),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.ChannelShuffle(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 이미지넷 데이터셋 통계값으로 Normalize
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 텐서타입은 안해줌
    ToTensorV2() # Normalize를 먼저하고 tensor화를 진행해야한다.
])

In [ ]:
## Pytorch BatchLoader 생성 (학습에 이용할 최종 dataloader)
from torch.utils.data import DataLoader as DataLoader

train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

valid_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transforms)
valid_loader = DataLoader(valid_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
train_dataset.__getitem__(0)

(tensor([[[ 0.2624,  0.2967,  0.3652,  ...,  0.0398, -0.1314,  0.0912],
          [ 0.3823,  0.3994,  0.3823,  ..., -0.0801, -0.2684,  0.2967],
          [ 0.4337,  0.3994,  0.2111,  ...,  0.0569,  0.0741,  0.2796],
          ...,
          [-0.2171, -0.1143, -0.0972,  ..., -0.3027, -0.1828, -0.1828],
          [-0.1314, -0.1828, -0.1999,  ..., -0.1143, -0.1999, -0.1314],
          [-0.1828, -0.1999, -0.0287,  ..., -0.1486, -0.1828, -0.0801]],
 
         [[ 0.3978,  0.4328,  0.5028,  ...,  0.1702, -0.0049,  0.2227],
          [ 0.5203,  0.5378,  0.5203,  ...,  0.0476, -0.1450,  0.4328],
          [ 0.5728,  0.5378,  0.3452,  ...,  0.1877,  0.2052,  0.4153],
          ...,
          [-0.0924,  0.0126,  0.0301,  ..., -0.1800, -0.0574, -0.0574],
          [-0.0049, -0.0574, -0.0749,  ...,  0.0126, -0.0749, -0.0049],
          [-0.0574, -0.0749,  0.1001,  ..., -0.0224, -0.0574,  0.0476]],
 
         [[ 0.6182,  0.6531,  0.7228,  ...,  0.3916,  0.2173,  0.4439],
          [ 0.7402,  0.7576,

# Model Define

In [ ]:
from torchvision import models # 모델 라이브러리 함수

resnet_50 = models.resnet50(pretrained=True).to(device) # 선행학습 여부

# resnet = models.resnet50(pretrained = True)


# finetuning
import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
resnet_50.fc = nn.Linear(resnet_50.fc.in_features, 19).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 213MB/s]


In [ ]:
from torchsummary import summary # 모델 아키텍쳐 확인하는 함수

summary(resnet_50, input_size = (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
optimizer = torch.optim.Adam(resnet_50.parameters(), lr = CFG['LEARNING_RATE'], weight_decay = 1e-8)

In [ ]:
from tqdm import tqdm
from tensorflow import summary
from torch.utils.tensorboard import SummaryWriter
import argparse
import logging
from pathlib import Path
from torch import optim
import numpy as np

train_acc_lst, train_loss_lst, test_acc_lst, test_loss_lst= [], [], [], []

epochs = 50
model_name = 'resnet50'
state={}

for epoch in tqdm(range(1, epochs+1)):

  train_loss = 0.0
  total = 0
  correct = 0
  train_acc = 0
  
  resnet_50.train()
  for i, (train_img, train_label) in enumerate(train_loader):
    # gpu에 할당
    train_img = train_img.to(device)
    train_label = train_label.to(device)

    output = resnet_50(train_img) # 모델에 입력

    optimizer.zero_grad( set_to_none = True ) # 계산했던 가중치 초기화    
    loss = criterion(output, train_label)
    loss.backward() # 미분
    optimizer.step() # 학습

    # loss & acc
    train_loss += loss.item()
    _, predictions = torch.max(output.data ,dim = 1)
   
    total += train_label.size(0)
    correct += (predictions == train_label).sum().item()
    train_acc += 100 * (correct / total)

  train_loss = round(train_loss/(i+1), 3) # 소수점 반올림
  train_acc = round(train_acc/(i+1), 3)
  print(f'Trainset {epoch}/{epochs} Loss : {train_loss}, Accuracy : {train_acc}%')
  train_acc_lst.append(train_acc)
  train_loss_lst.append(train_loss)

  # -------------------------------------------------------------------------------------
  test_loss = 0.0
  corrects = 0
  totals = 0
  test_acc = 0

  resnet_50.eval()
  for i, (valid_img, valid_label) in enumerate(valid_loader):
        # gpu에 할당
    valid_img = valid_img.to(device)
    valid_label = valid_label.to(device)

    outputs = resnet_50(valid_img) # 모델에 입력
    losses = criterion(outputs, valid_label)

    # loss & acc
    test_loss += losses.item()
    _, predictions = torch.max(outputs.data ,dim = 1 )
   
    totals += valid_label.size(0)
    corrects += (predictions == valid_label).sum().item()
    test_acc += 100 * (corrects / totals)

  test_loss = round(test_loss/(i+1), 3) # 소수점 반올림
  test_acc = round(test_acc/(i+1), 3)
  print(f'Validset {epoch}/{epochs} Loss : {test_loss}, Accuracy : {test_acc}% \n')
  test_loss_lst.append(test_loss)
  test_acc_lst.append(test_acc)



  if np.max(test_acc_lst) <= test_acc:
    state['epoch'] = epoch
    state['net'] = resnet_50.state_dict()


    state['test_loss'] = test_loss

    state['train_acc'] = train_acc
    state['test_acc'] = test_acc
torch.save(state, 'resnet50_submition_{}_{}.pth'.format(str(state['epoch']), str(state['test_acc'])))

  0%|          | 0/50 [00:00<?, ?it/s]

Trainset 1/50 Loss : 1.989, Accuracy : 35.482%


  2%|▏         | 1/50 [15:09<12:22:41, 909.42s/it]

Validset 1/50 Loss : 2.003, Accuracy : 45.576% 

Trainset 2/50 Loss : 1.86, Accuracy : 42.264%


  4%|▍         | 2/50 [16:44<5:44:13, 430.27s/it] 

...

Validset 48/50 Loss : 3.733, Accuracy : 62.912% 

Trainset 49/50 Loss : 0.664, Accuracy : 79.101%


 98%|█████████▊| 49/50 [1:29:06<01:33, 93.18s/it]

Validset 49/50 Loss : 1.84, Accuracy : 64.227% 

Trainset 50/50 Loss : 0.625, Accuracy : 78.827%


100%|██████████| 50/50 [1:30:40<00:00, 108.81s/it]

Validset 50/50 Loss : 1.685, Accuracy : 66.207% 



In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
import numpy as np

def label_decoder(l):
    new_label = [
        '가구수정' if p == 0 else
        '걸레받이수정' if p == 1 else
        '곰팡이' if p == 2 else
        '꼬임' if p == 3 else
        '녹오염' if p == 4 else
        '들뜸' if p == 5 else
        '면불량' if p == 6 else
        '몰딩수정' if p == 7 else
        '반점' if p == 8 else
        '석고수정' if p == 9 else
        '오염' if p == 10 else
        '오타공' if p == 11 else
        '울음' if p == 12 else
        '이음부불량' if p == 13 else
        '창틀,문틀수정' if p == 14 else
        '터짐' if p == 15 else
        '틈새과다' if p == 16 else
        '피스' if p == 17 else
        '훼손' if p == 18 else
        'NaN' for p in l
    ]
    return np.array(new_label)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = label_decoder(preds)
    return preds

In [ ]:
preds = inference(resnet_50, test_loader, device)

100%|██████████| 198/198 [03:30<00:00,  1.06s/it]


# submission

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/CV/도배하자유형분류경진대회/sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,오염
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [ ]:
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [ ]:
submit.to_csv('resnet50_submission.csv', index=False, encoding='utf-8')